In [11]:
import os, sys, glob, json, scipy as sp
import matplotlib.pyplot as plt
os.chdir("../")
path = os.getcwd()
sys.path.insert(0, path)
import GWGen
from GWGen.WFGenerator import *
from GWGen.DressedFluxes.Fluxes import *
import pyAnalyticFluxes as flux

etest = 0.1
ptest = 10
M=1e7*unit.M_sun
m=1*unit.M_sun
mu = 9e-19
a = 0.9

In [2]:
pc = ProcaSolution(M.value,a,mu)
consch = pc.ChangeInOrbitalConstants()
pn = PN(M.value,m.value, DeltaEFlux = consch["E"], DeltaLFlux = consch["L"])

UltralightBoson instance not provided. Instantiating class...
done.


In [12]:
#SI Units
dedeSI =(pn.dEde()(etest,ptest)*(m*cons.c**2)).decompose()
dedpSI = (pn.dEdp()(etest,ptest)*(m*cons.c**2 / (cons.G*M/(cons.c**2)))).decompose()
dldeSI = (pn.dLde()(etest,ptest)*(m*cons.G*M/(cons.c**2) * cons.c)).decompose()
dldpSI = (pn.dLdp()(etest,ptest)*(m*cons.G*M/(cons.c**2) * cons.c / (cons.G*M/(cons.c**2)))).decompose()
normSI = (dldpSI*dedeSI  - dldeSI*dedpSI)

ENsi = (flux.pydEdt(0.9,ptest,etest,1,10,10)*pn.epsilon**2) *cons.c**5 / cons.G
LNsi = ((flux.pydLdt(0.9,ptest,etest,1,10,10)*pn.epsilon*m)  * cons.c**2).decompose()

edotSI = (dldpSI*ENsi - dedpSI*LNsi)/normSI
print("From jacobian and (E,L) fluxes: ", edotSI.to(unit.Hz).value)
print("From pyAnalyticFluxes:               ", (flux.pydedt(a,ptest, etest,1,10,8)*pn.epsilon * cons.c**3/(cons.G*M)).to(unit.Hz).value)

From jacobian and (E,L) fluxes:  -3.6550576036603275e-13
From pyAnalyticFluxes:                -3.603198466207921e-13


In [13]:
#Geometric Units
dedegeo =(pn.dEde()(etest,ptest)*m)
dedpgeo = (pn.dEdp()(etest,ptest)*m)
dldegeo = (pn.dLde()(etest,ptest)*m*M)
dldpgeo = (pn.dLdp()(etest,ptest)*m*M)
normgeo = (dldpgeo*dedegeo  - dldegeo*dedpgeo)

ENgeo = flux.pydEdt(0.9,ptest,etest,1,10,10)*pn.epsilon**2*M
LNgeo = (flux.pydLdt(0.9,ptest,etest,1,10,10)*pn.epsilon*m*M)

edotgeo = (dldpgeo*ENgeo - dedpgeo*LNgeo)/normgeo
pdotgeo = (dedegeo*LNgeo - dldegeo*ENgeo)/normgeo
print("Eflux: ", ENgeo)
print("Lflux: ", LNgeo)
print("\nedot From jacobian and (E,L) fluxes: ", edotgeo)
print("edot From pyAnalyticFluxes:          ", flux.pydedt(a,ptest, etest,1,10,8)*pn.epsilon)
print("edot From PN class:                  ", pn.UndressedeFlux(etest,ptest))
print("\npdot From jacobian and (E,L) fluxes: ", pdotgeo)
print("pdot From pyAnalyticFluxes:          ", flux.pydpdt(a,ptest, etest,1,10,10)*pn.epsilon)
print("pdot From PN class:                  ", pn.UndressedpFlux(etest,ptest))

Eflux:  -5.134339169670736e-12 solMass
Lflux:  -0.0016351394946417083 solMass2

edot From jacobian and (E,L) fluxes:  -1.8002953139936168e-11
edot From pyAnalyticFluxes:           -1.7747521427862013e-11
edot From PN class:                   -1.7747521427862013e-11

pdot From jacobian and (E,L) fluxes:  -1.0948540664256946e-09
pdot From pyAnalyticFluxes:           -1.12405604548241e-09
pdot From PN class:                   -1.12405604548241e-09


In [14]:
print("_____ SI Units______")
print("E:")
ProcaENSI = pc.ChangeInOrbitalConstants()["E"](0,etest,ptest)
ProcaLNSI = pc.ChangeInOrbitalConstants()["L"](0,etest,ptest)
print(ProcaENSI)
print(ENsi)
print("L:")
print(ProcaLNSI)
print(LNsi)

print("\n\n")
print("_____ Geometric Units______")
print("E:")
inverseenergyflux = (cons.G/(cons.c**5))
energyfluxunit = (unit.kg*unit.m**2 / (unit.s**3))
ProcaENGeo = ((pc.ChangeInOrbitalConstants()["E"](0,etest,ptest)*energyfluxunit*inverseenergyflux*M).to(unit.M_sun))
print("Proca: ", ProcaENGeo)
print("GW:    ", ENgeo)
print("L:")
inverseangularmomentumflux = (1/(cons.c**2)).value
KGtoMsun = unit.kg.to(unit.Msun)
ProcaLNGeo = ((pc.ChangeInOrbitalConstants()["L"](0,etest,ptest)*KGtoMsun*inverseangularmomentumflux*M.value))
print("Proca: ", ProcaLNGeo)
print("GW:    ", LNgeo.value)

print("\n*** edot:")
print("Proca: ", (dldpgeo*ProcaENGeo - dedpgeo*ProcaLNGeo*unit.M_sun**2)/normgeo)
print("GW:    ", (dldpgeo*ENgeo - dedpgeo*LNgeo)/normgeo)
print("pyAnalyticFlux: ", flux.pydedt(a,ptest,etest,1,10,10)*pn.epsilon)

print("\n*** pdot:")
print("Proca: ", (dedegeo*ProcaLNGeo*unit.M_sun**2 - dldegeo*ProcaENGeo)/normgeo)
print("GW:    ", (dedegeo*LNgeo - dldegeo*ENgeo)/normgeo)
print("pyAnalyticFlux: ", flux.pydpdt(a,ptest, etest,1,10,8)*pn.epsilon)

print("\n*** from PN class instance")
print("pdot:     ", pn.UndressedpFlux(etest,ptest))
print("pdotCorr: ", (dedegeo*pn.LFluxModification(0, etest, ptest)*unit.Msun**2 - dldegeo*pn.EFluxModification(0,etest,ptest)*unit.Msun)/normgeo)
print("edot:     ", pn.UndressedeFlux(etest,ptest))
print("edotCorr: ", (dldpgeo*pn.EFluxModification(0, etest, ptest)*unit.Msun - dedpgeo*pn.EFluxModification(0,etest,ptest)*unit.Msun**2)/normgeo)

_____ SI Units______
E:
3.6823208530169655e+37
-1.8628691273268792e+34 kg m2 / s3
L:
3.373241742732122e+39
-2.9221474383830117e+37 kg m2 / s2



_____ Geometric Units______
E:
Proca:  1.014901364438301e-08 solMass
GW:     -5.134339169670736e-12 solMass
L:
Proca:  0.1887557323790437
GW:     -0.0016351394946417083

*** edot:
Proca:  1.1453308548028013e-06
GW:     -1.8002953139936168e-11
pyAnalyticFlux:  -1.7747521427860087e-11

*** pdot:
Proca:  -6.949004397852962e-08
GW:     -1.0948540664256946e-09
pyAnalyticFlux:  -1.1240560454828328e-09

*** from PN class instance
pdot:      -1.12405604548241e-09
pdotCorr:  -6.949004397852962e-08
edot:      -1.7747521427862013e-11
edotCorr:  1.214158384340145e-06
